# Finetuning model

## Testing Fine-tuning

## Transforming dataset :

In [66]:
import pandas as pd
csv = "/content/drive/MyDrive/UH - Final Year Project/data/df-iter-1.csv"
df = pd.read_csv(csv)
df

,sentence,vars,names,variable_position
0,The loading term becomes relevant at a time τ,['τ'],a time,['end']
1,we get easily for the rupture force f ∗,['f'],the rupture force,['end']
2,Green’s theorem can be used to show that the v...,['d3xG(x)'],the volume,['end']
3,at a ﬁxed point x,['x'],['point'],['end']
4,it will be quite small (|K| is large) around t...,['x'],the point,['end']
...,...,...,...,...
296,we are inspired by the recent advances in the ...,['(information)'],quantum,['end']
297,"then the correlation function C(A, B)","['C(A, B)']",the correlation function,['end']
298,Next is to utilize the perturbation expansion ...,['f'],the longitudinal distribution function,['end']
299,the resonator voltage V f,['V'],the resonator voltage,['end']


In [67]:
# drop unecessary "[]" inside names
df["names"] = df["names"].apply(lambda name: name.strip("[]'"))
df['vars'] = df['vars'].apply(lambda x: x.strip("[]'"))
df.drop(columns=["variable_position"],inplace=True)

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  301 non-null    object
 1   vars      301 non-null    object
 2   names     301 non-null    object
dtypes: object(3)
memory usage: 7.2+ KB


In [69]:
df

,sentence,vars,names
0,The loading term becomes relevant at a time τ,τ,a time
1,we get easily for the rupture force f ∗,f,the rupture force
2,Green’s theorem can be used to show that the v...,d3xG(x),the volume
3,at a ﬁxed point x,x,point
4,it will be quite small (|K| is large) around t...,x,the point
...,...,...,...
296,we are inspired by the recent advances in the ...,(information),quantum
297,"then the correlation function C(A, B)","C(A, B)",the correlation function
298,Next is to utilize the perturbation expansion ...,f,the longitudinal distribution function
299,the resonator voltage V f,V,the resonator voltage


## Tokenization :

In [70]:
from spacy.tokenizer import Tokenizer
import re


def my_tokenizer(text) :

  # defining function pattern
  func_name = r"α-ωA-Za-zΑ-Ω0-9ℰℓℒℳøℂℕℙℚℝℤΓΔΛΞΠΣΦΨΩÅℏ∞∘∂∫∮∯∇αβγ∅"
  func_var = r"A-Za-zα-ωΑ-ΩℰℓℒℳøℂℕℙℚℝℤΓΔΛΞΠΣΦΨΩÅℏ∞∘∂∫∮∯∇αβγ∅"
  func_pattern = fr"(.|)([{func_name}]{{1,3}}(′|.|)\([{func_var}](,\s*[{func_var}])*\))"

  # getting math functions
  matches = re.findall(func_pattern, text)

  # getting only second-group matches (the functions)
  functions = [match[1] for match in matches]
  func_saver = iter(functions.copy())

  # Replace math functions with temporary markers
  for func in functions:
    text = text.replace(func, "[FUNC]")

  # Tokenize the rest of the text
  tokens = text.split()

  # Replace temporary markers with original functions
  new_tokens = [next(func_saver) if '[FUNC]' in token else token for token in tokens]

  return new_tokens

In [71]:
def create_spacy_tokenizer(nlp):
    def custom_tokenizer(text):
        tokens = my_tokenizer(text)
        return Doc(nlp.vocab, words=tokens)
    return custom_tokenizer

In [72]:
import spacy
from spacy.tokens import Doc

nlp = spacy.load("en_core_web_sm")
nlp.tokenizer = create_spacy_tokenizer(nlp)

In [73]:
def tokenize(text) :
  tokens= []
  doc = nlp(text)

  # Iterate over the tokens in the processed doc
  for token in doc:
    tokens.append(str(token))
  return tokens

In [74]:
# this returns tokens inside the list
df['tokenized_sentence'] = df['sentence'].apply(tokenize)

In [75]:
df.head()

,sentence,vars,names,tokenized_sentence
0,The loading term becomes relevant at a time τ,τ,a time,"[The, loading, term, becomes, relevant, at, a,..."
1,we get easily for the rupture force f ∗,f,the rupture force,"[we, get, easily, for, the, rupture, force, f, ∗]"
2,Green’s theorem can be used to show that the v...,d3xG(x),the volume,"[Green’s, theorem, can, be, used, to, show, th..."
3,at a ﬁxed point x,x,point,"[at, a, ﬁxed, point, x]"
4,it will be quite small (|K| is large) around t...,x,the point,"[it, will, be, quite, small, (|K|, is, large),..."


## Labelling :

In [76]:
sentence = ["The","quick","brown","fox" ,"jumps" ,"over","the","lazy","dog"]
word = "The quick brown fox"
word_tokened = word.split()
start = sentence.index(word_tokened[0])
end = sentence.index(word_tokened[-1])

for i in range(start,end+1) :
  if i == start :
    sentence[i] = "B-NAME"
  else :
    sentence[i] = "I-NAME"

In [77]:
def labeling(dataframe):
  labz = []
  for var , name , tokenized_sents in zip(dataframe["vars"],dataframe["names"],dataframe["tokenized_sentence"]) :
    # initilaising everything as oustide of entity
    labels = ['O']*len(tokenized_sents)
    # anonattating variable name
    if var in tokenized_sents :
      idx = tokenized_sents.index(var)
      labels[idx] = "B-VAR"

    tokenized_name = name.split()
    # dealing with names :
    if len(tokenized_name) > 1 :
      if tokenized_name[0] in  tokenized_sents :
        # getting name start index
        start_idx = tokenized_sents.index(tokenized_name[0])
        # getting name end index
        end_idx = tokenized_sents.index(tokenized_name[-1])

        # looping over the labels :
        for i in range(start_idx,end_idx+1) :
          if i == start_idx :
            labels[i] = "B-NAME"
          else :
            labels[i] = "I-NAME"
    elif len(tokenized_name) == 1 :
      if tokenized_name in tokenized_sents :
        idx = tokenized_sents.index(tokenized_name)
        labels[i] = "B-NAME"
    labz.append(labels)
  return labz

In [78]:
df["labels"] = labeling(df)
df

,sentence,vars,names,tokenized_sentence,labels
0,The loading term becomes relevant at a time τ,τ,a time,"[The, loading, term, becomes, relevant, at, a,...","[O, O, O, O, O, O, B-NAME, I-NAME, B-VAR]"
1,we get easily for the rupture force f ∗,f,the rupture force,"[we, get, easily, for, the, rupture, force, f, ∗]","[O, O, O, O, B-NAME, I-NAME, I-NAME, B-VAR, O]"
2,Green’s theorem can be used to show that the v...,d3xG(x),the volume,"[Green’s, theorem, can, be, used, to, show, th...","[O, O, O, O, O, O, O, O, B-NAME, I-NAME, O, O,..."
3,at a ﬁxed point x,x,point,"[at, a, ﬁxed, point, x]","[O, O, O, O, B-VAR]"
4,it will be quite small (|K| is large) around t...,x,the point,"[it, will, be, quite, small, (|K|, is, large),...","[O, O, O, O, O, O, O, O, O, B-NAME, I-NAME, B-..."
...,...,...,...,...,...
296,we are inspired by the recent advances in the ...,(information),quantum,"[we, are, inspired, by, the, recent, advances,...","[O, O, O, O, O, O, O, O, O, O, O, O, B-VAR, O]"
297,"then the correlation function C(A, B)","C(A, B)",the correlation function,"[then, the, correlation, function, C(A, B)]","[O, B-NAME, I-NAME, I-NAME, B-VAR]"
298,Next is to utilize the perturbation expansion ...,f,the longitudinal distribution function,"[Next, is, to, utilize, the, perturbation, exp...","[O, O, O, O, B-NAME, I-NAME, I-NAME, I-NAME, I..."
299,the resonator voltage V f,V,the resonator voltage,"[the, resonator, voltage, V, f]","[B-NAME, I-NAME, I-NAME, B-VAR, O]"


In [79]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
# testing on 1 sentence
input_ids = tokenizer.convert_tokens_to_ids(df["tokenized_sentence"].iloc[1])
input_ids

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

[185, 2744, 3717, 168, 111, 12571, 3163, 125, 3149]

In [80]:
# Convert DataFrame to Dataset
from datasets import Dataset
dataset = Dataset.from_pandas(df)

In [82]:
dataset

Dataset({
    features: ['sentence', 'vars', 'names', 'tokenized_sentence', 'labels'],
    num_rows: 301
})

In [83]:
label2id = {'O': 0, 'B-VAR': 1, 'B-NAME': 2, 'I-NAME': 3}
id2label = {v: k for k, v in label2id.items()}